
# Greedy Fleet Electrification Driver (Notebook) — v3

**What’s new in v3**
- Fixes `SameFileError` by skipping the copy when the source and destination are the same file.
- Keeps the v2 fixes (correct `nbclient` import + per‑round snapshots of `input_ICE.csv` and `cascades/`).


In [9]:

# If needed, install libraries (safe to run multiple times)
try:
    import nbclient, nbformat, pandas  # noqa: F401
except Exception:
    %pip install -q nbclient nbformat pandas


## Parameters

In [11]:

from pathlib import Path

# Directory that contains the three notebooks and where intermediate folders will be created.
WORKDIR = Path(".").resolve()

# Path to your starting fleet CSV (this can be outside WORKDIR).
# The driver will copy it to WORKDIR / WORKING_FLEET_FILENAME each round (unless it's already the same path).
INITIAL_FLEET = WORKDIR / "filtered_fleet_noGTPD.csv"  # <-- change if your file is elsewhere

# Safety cap on number of rounds
MAX_ROUNDS = 100

# Fresh run options
RESET_OUTPUT_FOLDERS_EACH_ROUND = True   # Clears 'All_Electric_Cascades' and 'Dept Savings Results' every round
RESET_GREEDY_ROUNDS_ON_START = True      # Clears 'Greedy_Rounds' at the very beginning

# Notebook filenames (must exist in WORKDIR)
CASCADE_NB = WORKDIR / "Cascade Generator.ipynb"
COST_NB    = WORKDIR / "Cost Modeller.ipynb"
APPLY_NB   = WORKDIR / "Apply_Top_Department_Cascade_v2.ipynb"

# The filename your notebooks expect for the working fleet
WORKING_FLEET_FILENAME = "filtered_fleet_noGTPD.csv"

# Expected outputs written by the Apply notebook (in WORKDIR)
APPLY_OUTPUTS = {
    "evs": "fleet_round1_top_dept_EVs.csv",
    "ice": "fleet_round1_remaining_ICE.csv",
    "log": "fleet_after_round1_change_log.csv",
}

# Output root for this driver
GREEDY_OUT = WORKDIR / "Greedy_Rounds"


## Utilities

In [12]:

import shutil, os
import json
import pandas as pd
import nbformat

from nbclient import NotebookClient
try:
    from nbclient.exceptions import CellExecutionError
except Exception:
    class CellExecutionError(Exception):
        pass

def run_notebook(nb_path: Path, timeout: int = 1800):
    if not nb_path.exists():
        raise FileNotFoundError(f"Notebook not found: {nb_path}")
    nb = nbformat.read(nb_path, as_version=4)
    client = NotebookClient(
        nb,
        timeout=timeout,
        kernel_name="python3",
        allow_errors=False,
        resources={"metadata": {"path": str(nb_path.parent)}},
    )
    try:
        client.execute()
    except CellExecutionError as e:
        raise RuntimeError(f"Execution failed in {nb_path.name}:\n{e}") from e

def safe_clean_dir(path: Path):
    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents=True, exist_ok=True)

def ensure_dir(path: Path):
    path.mkdir(parents=True, exist_ok=True)

def find_required_file(path: Path, name: str) -> Path:
    f = path / name
    if not f.exists():
        raise FileNotFoundError(f"Expected file not found: {f}")
    return f

def move_over(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    shutil.move(str(src), str(dst))

def copy_over(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(src, dst)

def paths_are_same(a: Path, b: Path) -> bool:
    """Robust 'same file' check that works even if one path doesn't exist yet."""
    try:
        return os.path.samefile(a, b)
    except FileNotFoundError:
        # If dest doesn't exist yet, fall back to resolved path equality
        try:
            return a.resolve() == b.resolve()
        except Exception:
            return str(a) == str(b)


## Run Greedy Rounds

In [13]:

# Prepare
if RESET_GREEDY_ROUNDS_ON_START:
    safe_clean_dir(GREEDY_OUT)
else:
    ensure_dir(GREEDY_OUT)

# Validate notebooks exist
for p in (CASCADE_NB, COST_NB, APPLY_NB):
    if not p.exists():
        raise FileNotFoundError(f"Missing notebook: {p}")

# Validate starting fleet exists
if not Path(INITIAL_FLEET).exists():
    raise FileNotFoundError(f"Initial fleet CSV not found: {INITIAL_FLEET}")

rounds_manifest = []
current_ice = Path(INITIAL_FLEET)

for r in range(1, MAX_ROUNDS + 1):
    print(f"\\n=== Round {r} ===")

    # 0) Write working fleet to expected filename in WORKDIR (skip copy if same file)
    working_fleet = WORKDIR / WORKING_FLEET_FILENAME
    if paths_are_same(current_ice, working_fleet):
        print(f"-> Working fleet already at {working_fleet}; skipping copy.")
    else:
        shutil.copy2(current_ice, working_fleet)
        print(f"-> Working fleet staged at: {working_fleet}")

    # 1) Reset handoff folders if requested
    cascades_dir = WORKDIR / "All_Electric_Cascades"
    savings_dir  = WORKDIR / "Dept Savings Results"
    if RESET_OUTPUT_FOLDERS_EACH_ROUND:
        safe_clean_dir(cascades_dir)
        safe_clean_dir(savings_dir)
        print("-> Reset 'All_Electric_Cascades' and 'Dept Savings Results'")

    # 2) Run Cascade Generator
    print("-> Running Cascade Generator...")
    run_notebook(CASCADE_NB)

    cascade_files = [p for p in cascades_dir.glob("*.csv") if "__log" not in p.name]
    if not cascade_files:
        print("No cascade files produced; stopping.")
        break

    # 3) Run Cost Modeller
    print("-> Running Cost Modeller...")
    run_notebook(COST_NB)

    dept_summary_path = find_required_file(savings_dir, "Department_Savings_Summary.csv")
    df_summ = pd.read_csv(dept_summary_path)
    if df_summ.empty or "Department" not in df_summ.columns:
        print("Department savings summary is empty or missing 'Department'; stopping.")
        break

    # Pick top department (sort if not already sorted)
    if "Total 10-Year Savings ($)" in df_summ.columns:
        df_summ = df_summ.sort_values("Total 10-Year Savings ($)", ascending=False, kind="mergesort")
    top_row = df_summ.iloc[0]
    top_dept = str(top_row["Department"])
    top_savings = float(top_row.get("Total 10-Year Savings ($)", 0.0))
    print(f"-> Top department: {top_dept} (10-yr savings ≈ ${top_savings:,.0f})")

    # 4) Apply cascades & EV replacement
    print("-> Applying top department cascades and EV conversion...")
    run_notebook(APPLY_NB)

    # Create round dir and snapshot the exact input ICE and cascades used this round
    round_dir = GREEDY_OUT / f"round_{r:02d}_{top_dept.replace(' ', '_')}"
    ensure_dir(round_dir)

    # Save the exact input ICE used at the START of this round
    copy_over(working_fleet, round_dir / "input_ICE.csv")

    # Snapshot cascades folder
    try:
        shutil.copytree(cascades_dir, round_dir / "cascades", dirs_exist_ok=True)
    except Exception as e:
        print(f"(warn) Failed to snapshot cascades folder: {e}")

    # Collect outputs from Apply notebook
    evs_csv = find_required_file(WORKDIR, APPLY_OUTPUTS["evs"])
    ice_csv = find_required_file(WORKDIR, APPLY_OUTPUTS["ice"])
    change_log_csv = find_required_file(WORKDIR, APPLY_OUTPUTS["log"])

    # Stash artifacts for this round
    move_over(evs_csv, round_dir / "top_dept_EVs.csv")
    move_over(ice_csv, round_dir / "remaining_ICE.csv")
    move_over(change_log_csv, round_dir / "change_log.csv")
    copy_over(dept_summary_path, round_dir / "dept_savings_summary.csv")

    try:
        n_evs = len(pd.read_csv(round_dir / "top_dept_EVs.csv"))
    except Exception:
        n_evs = None

    rounds_manifest.append({
        "round": r,
        "department": top_dept,
        "savings_10y_usd": top_savings,
        "n_top_dept_evs": n_evs,
        "round_folder": str(round_dir),
    })

    # 5) Prepare next round
    next_ice = round_dir / "remaining_ICE.csv"
    if not next_ice.exists():
        print("Remaining ICE dataset missing; stopping.")
        break

    df_next = pd.read_csv(next_ice)
    if df_next.empty:
        print("Remaining ICE dataset is empty; stopping.")
        break
    if "Department" not in df_next.columns:
        print("Remaining ICE dataset lacks 'Department' column; stopping.")
        break
    if df_next["Department"].nunique() == 0:
        print("No departments remain; stopping.")
        break

    current_ice = next_ice

# Write manifest
manifest_path = GREEDY_OUT / "rounds_manifest.json"
with open(manifest_path, "w", encoding="utf-8") as f:
    json.dump(rounds_manifest, f, indent=2)

print(f"\\nWrote manifest: {manifest_path}")
print("Greedy run complete.")


\n=== Round 1 ===
-> Working fleet already at /Users/asubramanian115/Documents/Cascading Pipeline v3/filtered_fleet_noGTPD.csv; skipping copy.
-> Reset 'All_Electric_Cascades' and 'Dept Savings Results'
-> Running Cascade Generator...
-> Running Cost Modeller...
-> Top department: Gtri (10-yr savings ≈ $216,580)
-> Applying top department cascades and EV conversion...
\n=== Round 2 ===
-> Working fleet staged at: /Users/asubramanian115/Documents/Cascading Pipeline v3/filtered_fleet_noGTPD.csv
-> Reset 'All_Electric_Cascades' and 'Dept Savings Results'
-> Running Cascade Generator...
-> Running Cost Modeller...
-> Top department: I&S (Fleet Rental) (10-yr savings ≈ $55,774)
-> Applying top department cascades and EV conversion...
\n=== Round 3 ===
-> Working fleet staged at: /Users/asubramanian115/Documents/Cascading Pipeline v3/filtered_fleet_noGTPD.csv
-> Reset 'All_Electric_Cascades' and 'Dept Savings Results'
-> Running Cascade Generator...
-> Running Cost Modeller...
-> Top departm

## Inspect Results (optional)

In [14]:

import json, pandas as pd
from pathlib import Path

manifest_path = GREEDY_OUT / "rounds_manifest.json"
if manifest_path.exists():
    with open(manifest_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if data:
        df_manifest = pd.DataFrame(data)
        display(df_manifest)
    else:
        print("Manifest exists but is empty.")
else:
    print("Manifest not found yet. Run the previous cell first.")


,round,department,savings_10y_usd,n_top_dept_evs,round_folder
0,1,Gtri,216580.36,16,/Users/asubramanian115/Documents/Cascading Pip...
1,2,I&S (Fleet Rental),55773.63,4,/Users/asubramanian115/Documents/Cascading Pip...
2,3,Gtri,55733.53,5,/Users/asubramanian115/Documents/Cascading Pip...
3,4,Parking,34354.10,5,/Users/asubramanian115/Documents/Cascading Pip...
4,5,Dining Services,33229.20,2,/Users/asubramanian115/Documents/Cascading Pip...
5,6,I&S (Bldg Services),26161.72,6,/Users/asubramanian115/Documents/Cascading Pip...
6,7,Ce,18092.36,3,/Users/asubramanian115/Documents/Cascading Pip...
7,8,Oit - Ne,12983.61,1,/Users/asubramanian115/Documents/Cascading Pip...
8,9,Housing,10430.66,11,/Users/asubramanian115/Documents/Cascading Pip...
9,10,Architecture,8582.04,1,/Users/asubramanian115/Documents/Cascading Pip...
